In [1]:
%load_ext autoreload
%autoreload 2
import scipy.stats as sps
from sklearn.model_selection import train_test_split
import gc
from sklearn.linear_model import LinearRegression
from dateutil.parser import *
import pandas as pd
import numpy as np
import os
from collections import Counter
import json
import typing as tp
from sklearn.preprocessing import normalize
import my_library as lib
from datetime import datetime
import re
import gensim
import gensim as gen
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import ast

In [ ]:
req = """
SELECT
    type,
    checker_function
FROM "//home/cloud_analytics/scoring_v2/data_tables/column_type_description"
FORMAT TabSeparatedWithNames
"""
type_df = lib.execute_query(req)

In [15]:
numeric_checker = """
def numeric_checker(df, column):
    df[column] = df[column].astype(float)
    return True
"""

In [19]:
binary_checker = """
def binary_checker(df, column):
    df[column] = df[column].astype(int)
    return set(df[column]) <= set([0, 1])
"""

In [20]:
category_checker = """
def category_checker(df, column):
    df[column] = df[column].astype(str)
    return df[column].unique().shape[0] < 100
"""

In [36]:
json_checker = """
def json_checker(df, column):
    import ast
    try:
        df[column].apply(lambda x: ast.literal_eval(x.replace("\\", "")))
        return True
    except Exception:
        return False
"""

json_pct_checker = """
def json_pct_checker(df, column):
    import ast
    try:
        df[column].apply(lambda x: ast.literal_eval(x.replace("\\", "")))
        return True
    except Exception:
        return False
"""

In [42]:
binary_target_checker = """
def binary_target_checker(df, column):
    df[column] = df[column].astype(int)
    return set(df[column]) <= set([0, 1])
"""

In [43]:
df = pd.DataFrame([
    ["numeric", numeric_checker],
    ['binary', binary_checker],
    ["category", category_checker],
    ["json", json_checker],
    ['json_pct', json_pct_checker],
    ['binary_target', binary_target_checker]
], columns=["type", "checker_function"])

In [44]:
lib.save_table("column_type_description", 
               "//home/cloud_analytics/scoring_v2/data_tables",
               df)